In [1]:
import h5py
import numpy as np

from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score

from neuropacks import ECOG

In [2]:
ecog = ECOG(data_path='/Users/psachdeva/data/ECoG/data/r32_b7.mat')

In [3]:
# load design matrix
X = ecog.get_design_matrix(
    form='abf',
    n_gaussians=7
)

# load responses
Y = ecog.get_response_matrix(
    bounds=(40, 60),
    band='HG',
    electrodes=None,
    transform=None
)

In [2]:
# load ecog object
ecog = ECOG(data_path='/Users/psachdeva/data/ECoG/data/r32_b7.mat')
# load fits
fits = h5py.File('fits/ecog/ecog.h5', 'r')

In [3]:
# load design matrix
X = ecog.get_design_matrix(
    form='bf',
    n_gaussians=7
)

In [4]:
# load responses
Y = ecog.get_response_matrix(
    bounds=(40, 60),
    band='HG',
    electrodes=None,
    transform=None
)

In [4]:
for i in range(Y.shape[1]):
    # fit lasso object
    lasso = LassoCV(
        normalize=False,
        cv=10,
        max_iter=100000
    )
    lasso.fit(X, Y[:, i])

In [5]:
# choose an electrode and fold
electrode = 12
fold = 3

In [6]:
# extract training and test sets
training_idx = fits['HG_bf_Lasso/training_folds/fold_%g' %fold][:]
test_idx = fits['HG_bf_Lasso/test_folds/fold_%g' %fold][:]
X_train = X[training_idx, :]
y_train = Y[training_idx, :][:, electrode]

In [7]:
# fit lasso object
lasso = LassoCV(
    normalize=True,
    cv=10,
    max_iter=100000
)

In [8]:
lasso.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=100000, n_alphas=100, n_jobs=None, normalize=True,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [9]:
# compare tuning coefficients
print(lasso.coef_)
print(fits['HG_bf_Lasso/tuning_coefs'][fold, electrode, :])

[-0.18590894 -0.          0.          0.          0.          0.90709511
  0.81805925]
[-0.18590894 -0.          0.          0.          0.          0.90709511
  0.81805925]


In [10]:
# compare intercepts
print(lasso.intercept_)
print(fits['HG_bf_Lasso/intercepts'][fold, electrode])

0.6822950387118683
0.6822950387118683


In [11]:
# compare test scores
X_test = X[test_idx, :]
y_test = Y[test_idx, :][:, electrode]

print(r2_score(y_test, np.dot(X_test, lasso.coef_) + lasso.intercept_))
print(fits['HG_bf_Lasso/r2'][fold, electrode])

0.05234734181109524
0.05234734181109524
